In [ ]:
!poetry update -q

In [ ]:
from configurations.cypher_chat_openai import CypherChatOpenAIConfiguration
from configurations.neo4j import Neo4jConfiguration
from configurations.configuration import Configuration
from configurations.chat_openai import ChatOpenAIConfiguration

configurations = Configuration.load('../configuration.yaml')
neo4j = Neo4jConfiguration.grab(configurations)
chat_openai = ChatOpenAIConfiguration.grab(configurations)
cypher_chat_openai = CypherChatOpenAIConfiguration.grab(configurations)

In [ ]:
from frames_benchmark_record import FramesBenchmarkRecord
from load_csv import load_csv

tests = load_csv('test.tsv', FramesBenchmarkRecord, delimiter='\t')
with open('imported_links.txt', 'r') as f:
    imported_links = [line.strip() for line in f.readlines()]
tests_for_imported: list[FramesBenchmarkRecord] = list(filter(lambda t: all((wiki_link in imported_links) for wiki_link in t.wiki_links), tests))

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

graph = Neo4jGraph(**neo4j.dict())
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
    Schema:
    {schema}
    Note: Do not include any explanations or apologies in your responses.
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    Do not include any text except the generated Cypher statement.
    
    The question is:
    {question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(input_variables=['schema', 'question'], validate_template=True, template=CYPHER_GENERATION_TEMPLATE)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(**cypher_chat_openai.dict()),
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

response = chain.invoke({'query': tests_for_imported[0].prompt})
response

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
prompt = """===Task===

I need your help in evaluating an answer provided by an LLM against a ground truth answer. Your task is to determine if the ground truth answer is present in the LLM’s response. Please analyze the provided data and make a decision.
===Instructions===
1. Carefully compare the "Predicted Answer" with the "Ground Truth Answer".
2. Consider the substance of the answers – look for equivalent information or correct answers. Do not focus on exact wording unless the exact wording is crucial to the meaning.
3. Your final decision should be based on whether the meaning and the vital facts of the "Ground Truth Answer" are present in the "Predicted Answer:"

===Input Data===
- Question: «{question}»
- Predicted Answer: «{answer}»
- Ground Truth Answer: «{ground_truth}»

===Output Format===
Provide your final evaluation in the following format:
"Explanation:" (How you made the decision?)
"Decision:" ("TRUE" or "FALSE" )

Please proceed with the evaluation. """

chat = ChatOpenAI(**chat_openai.dict())
message = chat.invoke([
    SystemMessage(content=prompt),
    HumanMessage(content=tests_for_imported[0].prompt)
])
print(message.content)

In [ ]:
# for t in tests:
#     if all((wiki_link in imported_links) for wiki_link in t.wiki_links):
#         print(t.id)